In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set()

1. Regarder toutes les features et leurs caractéristiques :
    * Nom
    * Type
    * % de valeurs manquantes
    * Bruit
    * Utilité pour le projet 
    * Distribution (gaussien, uniforme…)
2. Visualiser les données
3. Regarder les corrélations entre les features
4. Identifier les transformations / créations / ajouts de features possibles 

1. Nettoyer les données
    * S’occuper des outliers (supprimer, modifier…)
    * S’occuper des valeurs manquantes (remplir avec 0, moyenne, médiane, supprimer les exemples associés…)
2. Sélectionner les features utiles et supprimer les features inutiles pour le projet
3. Faire du Feature engineering
    * Echantilloner  les features continues
    * Décomposer les features catégorielles, date/heure…
    * Appliquer les transformations identifiées (log, sqrt, ^2…)
    * Créer de nouvelles features 
4. Normaliser ou standardiser les features 

In [117]:
path = '/Users/jeremieperes/MS_Big_data_Telecom/P1/INFMDI721_Kit_Data_Sciences/HackatonMDI720'

urban_population = pd.read_csv(f'{path}/data/Urban-population-by-country.csv', skiprows=4)
population_growth = pd.read_csv(f'{path}/data/Population-growth-by-country.csv', skiprows=4)
population = pd.read_csv(f'{path}/data/Population-by-country.csv', skiprows=4)
access_electricity = pd.read_csv(f'{path}/data/Access-to-electricity-by-country.csv', skiprows=4)

In [8]:
urban_population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,Urban population,SP.URB.TOTL,27526.0,28141.0,28532.0,28761.0,28924.0,29082.0,29253.0,29416.0,29575.0,29738.0,29900.0,30082.0,30275.0,30470.0,30605.0,30661.0,30615.0,30495.0,30353.0,30282.0,30332.0,...,40232.0,40970.0,41488.0,41945.0,42444.0,43048.0,43670.0,44246.0,44669.0,44889.0,44882.0,44686.0,44378.0,44053.0,43778.0,43819.0,44057.0,44348.0,44665.0,44979.0,45296.0,45616.0,45948.0,NaN,NaN
1,Afghanistan,AFG,Urban population,SP.URB.TOTL,755836.0,796272.0,839385.0,885228.0,934135.0,986074.0,1041191.0,1099272.0,1161355.0,1228273.0,1300947.0,1379463.0,1463290.0,1551037.0,1640868.0,1730928.0,1821611.0,1912078.0,1997578.0,2070935.0,2136374.0,...,4093835.0,4220821.0,4321584.0,4434762.0,4587798.0,4790053.0,5031157.0,5293385.0,5563504.0,5824291.0,6055019.0,6263747.0,6464835.0,6680732.0,6927764.0,7212518.0,7528588.0,7865067.0,8204877.0,8535606.0,8852859.0,9164841.0,9477100.0,NaN,NaN
2,Angola,AGO,Urban population,SP.URB.TOTL,569222.0,597288.0,628381.0,660180.0,691532.0,721552.0,749534.0,776116.0,804107.0,837758.0,881022.0,944294.0,1028199.0,1124624.0,1230706.0,1343551.0,1462599.0,1588714.0,1723460.0,1868827.0,2026766.0,...,6530150.0,6918998.0,7328066.0,7758423.0,8212001.0,8688765.0,9190861.0,9721267.0,10284528.0,10882817.0,11437890.0,12025601.0,12644649.0,13291081.0,13963065.0,14660282.0,15383127.0,16130304.0,16900847.0,17691524.0,18502165.0,19332881.0,20184707.0,NaN,NaN
3,Albania,ALB,Urban population,SP.URB.TOTL,493982.0,513592.0,530766.0,547928.0,565248.0,582374.0,599300.0,616687.0,635924.0,656733.0,677801.0,698647.0,720649.0,742333.0,764166.0,786668.0,809052.0,832109.0,854618.0,876974.0,902120.0,...,1250518.0,1260414.0,1270214.0,1279853.0,1289391.0,1298584.0,1327220.0,1354848.0,1381828.0,1407298.0,1430886.0,1452398.0,1473392.0,1495260.0,1519519.0,1546929.0,1575788.0,1603505.0,1630119.0,1654503.0,1680247.0,1706345.0,1728969.0,NaN,NaN
4,Andorra,AND,Urban population,SP.URB.TOTL,7839.0,8766.0,9754.0,10811.0,11915.0,13067.0,14262.0,15494.0,16765.0,18083.0,19458.0,20902.0,22401.0,23917.0,25397.0,26791.0,28085.0,29304.0,30491.0,31774.0,33205.0,...,60128.0,59940.0,59605.0,59649.0,60417.0,61991.0,64194.0,66747.0,69192.0,71205.0,72737.0,73843.0,74641.0,75098.0,75007.0,74305.0,73056.0,71515.0,70057.0,68919.0,68213.0,67876.0,67813.0,NaN,NaN


In [9]:
population_growth.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,Population growth (annual %),SP.POP.GROW,3.148037,2.238144,1.409622,0.832453,0.592649,0.573468,0.616991,0.587373,0.568530,0.580642,0.572211,0.636273,0.670693,0.671205,0.471968,0.212898,-0.117120,-0.363781,-0.436628,-0.204858,0.193211,...,3.517889,2.669576,2.114386,1.960558,2.055027,2.225930,2.229056,2.109354,1.757353,1.302039,0.799546,0.384052,0.134268,0.095655,0.210709,0.370125,0.502430,0.582349,0.594397,0.544892,0.507618,0.469944,0.453576,NaN,NaN
1,Afghanistan,AFG,Population growth (annual %),SP.POP.GROW,1.828639,1.898476,1.965751,2.029893,2.090248,2.147567,2.171009,2.188108,2.254572,2.384720,2.536599,2.665192,2.714523,2.659116,2.479436,2.200780,1.981420,1.747839,1.281617,0.522260,-0.407546,...,4.019468,2.636539,1.947319,2.170439,2.975053,3.902806,4.496720,4.668361,4.321539,3.682732,2.990478,2.493842,2.268277,2.397021,2.746576,3.143126,3.407587,3.494589,3.355582,3.077084,2.778317,2.548347,2.384761,NaN,NaN
2,Angola,AGO,Population growth (annual %),SP.POP.GROW,1.369926,1.393363,1.383629,1.256555,0.973962,0.617544,0.184283,-0.120653,-0.044882,0.546045,1.489917,2.521468,3.381711,3.898493,3.989233,3.810585,3.573056,3.432752,3.354068,3.374209,3.454406,...,3.214234,3.217320,3.228941,3.249247,3.277204,3.301198,3.329257,3.378811,3.453014,3.537557,3.619575,3.680635,3.710531,3.703878,3.671462,3.634159,3.597774,3.551950,3.497493,3.438851,3.378273,3.322081,3.276134,NaN,NaN
3,Albania,ALB,Population growth (annual %),SP.POP.GROW,3.015423,3.120855,3.056731,2.953749,2.880686,2.754021,2.634564,2.630190,2.842511,2.896083,2.550851,2.422972,2.494973,2.362552,2.297214,2.301154,2.208235,2.213252,2.075742,1.989457,2.047964,...,-0.621511,-0.625430,-0.629334,-0.633352,-0.637357,-0.938470,-0.299877,-0.374149,-0.417931,-0.511790,-0.630911,-0.755719,-0.767343,-0.673894,-0.496462,-0.269017,-0.165151,-0.183211,-0.207047,-0.291206,-0.159880,-0.091972,-0.246732,NaN,NaN
4,Andorra,AND,Population growth (annual %),SP.POP.GROW,7.050663,6.941532,6.692697,6.559522,6.241511,5.998800,5.750878,5.500706,5.309820,5.198287,5.147543,5.150130,5.083934,4.862724,4.461191,3.935751,3.431725,3.080122,2.905118,3.153934,3.524383,...,0.795574,-0.051287,-0.288007,0.354831,1.572166,2.939992,3.942573,4.375449,4.098923,3.382417,2.659984,2.066338,1.414648,0.714098,-0.016577,-0.834745,-1.588730,-2.025792,-1.951470,-1.529058,-0.919470,-0.383674,0.006493,NaN,NaN


In [10]:
population.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,59063.0,59440.0,59840.0,60243.0,60528.0,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,...,83200.0,85451.0,87277.0,89005.0,90853.0,92898.0,94992.0,97017.0,98737.0,100031.0,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,10174836.0,10399926.0,10637063.0,10893776.0,11173642.0,11475445.0,11791215.0,12108963.0,12412950.0,12689160.0,12943093.0,13171306.0,13341198.0,13411056.0,13356511.0,...,18853437.0,19357126.0,19737765.0,20170844.0,20779953.0,21606988.0,22600770.0,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,5781214.0,5774243.0,5771652.0,5803254.0,5890365.0,6040777.0,6248552.0,6496962.0,6761380.0,7024000.0,7279509.0,7533735.0,7790707.0,8058067.0,8341289.0,...,14400719.0,14871570.0,15359601.0,15866869.0,16395473.0,16945753.0,17519417.0,18121479.0,18758145.0,19433602.0,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,...,3168033.0,3148281.0,3128530.0,3108778.0,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,24276.0,25559.0,26892.0,28232.0,29520.0,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,...,64360.0,64327.0,64142.0,64370.0,65390.0,67341.0,70049.0,73182.0,76244.0,78867.0,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,NaN,NaN


In [11]:
access_electricity.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,100.0,100.0,100.0,91.660398,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,93.356292,100.000000,100.000000,100.000000,100.0,100.0,100.000000,100.00000,NaN,NaN,NaN
1,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,26.929317,33.015751,42.400000,45.237122,42.700000,43.222019,69.100000,70.153481,89.5,71.5,97.700000,97.70000,NaN,NaN,NaN
2,Angola,AGO,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20.0,25.073256,26.145882,27.194372,28.216885,29.223904,37.500000,31.245735,32.287685,33.368420,34.600000,35.670788,36.878685,32.0,42.0,40.624527,41.88623,NaN,NaN,NaN
3,Albania,ALB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,100.0,100.0,100.0,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.000000,100.00000,NaN,NaN,NaN
4,Andorra,AND,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,100.0,100.0,100.0,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.000000,100.00000,NaN,NaN,NaN


In [48]:
urban_population.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263
Country Name,Aruba,Afghanistan,Angola,Albania,Andorra,Arab World,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,Australia,Austria,Azerbaijan,Burundi,Belgium,Benin,Burkina Faso,Bangladesh,Bulgaria,Bahrain,"Bahamas, The",Bosnia and Herzegovina,Belarus,Belize,...,Sub-Saharan Africa (IDA & IBRD countries),Trinidad and Tobago,Tunisia,Turkey,Tuvalu,Tanzania,Uganda,Ukraine,Upper middle income,Uruguay,United States,Uzbekistan,St. Vincent and the Grenadines,"Venezuela, RB",British Virgin Islands,Virgin Islands (U.S.),Vietnam,Vanuatu,World,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
Country Code,ABW,AFG,AGO,ALB,AND,ARB,ARE,ARG,ARM,ASM,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BFA,BGD,BGR,BHR,BHS,BIH,BLR,BLZ,...,TSS,TTO,TUN,TUR,TUV,TZA,UGA,UKR,UMC,URY,USA,UZB,VCT,VEN,VGB,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
Indicator Name,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,...,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population,Urban population
Indicator Code,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,...,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL,SP.URB.TOTL
1960,27526,755836,569222,493982,7839,2.87972e+07,67927,1.50768e+07,960956,13324,21466,8.37831e+06,4.56117e+06,2.05143e+06,58113,8.46332e+06,225533,226977,2.46549e+06,2.9188e+06,133710,65405,614167,2.65623e+06,49740,...,3.33808e+07,289627,1.5673e+06,8.65791e+06,846,527336,298903,1.99636e+07,3.34643e+08,2.03703e+06,1.26462e+08,2.89707e+06,21008,5.01643e+06,984,18355,4.8025e+06,6626,1.01872e+09,20559,NaN,483697,7.97177e+06,557192,476164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016,45296,8.85286e+06,1.85022e+07,1.68025e+06,68213,2.35729e+08,8.04717e+06,3.99405e+07,1.85218e+06,48605,23486,2.07558e+07,5.05897e+06,5.36885e+06,1.29925e+06,1.10956e+07,5.02605e+06,5.24597e+06,5.54209e+07,5.29804e+06,1.27024e+06,313048,1.60909e+06,7.37899e+06,167604,...,3.99973e+08,733553,7.7258e+06,5.9175e+07,6807,1.71529e+07,8.96985e+06,3.11225e+07,1.69036e+09,3.25786e+06,2.64473e+08,1.6131e+07,56232,2.63139e+07,13784,102651,3.23147e+07,69755,4.03654e+09,36341,NaN,9.61592e+06,3.6724e+07,6.94434e+06,4.53126e+06
2017,45616,9.16484e+06,1.93329e+07,1.70634e+06,67876,2.41544e+08,8.18252e+06,4.04107e+07,1.85826e+06,48484,23583,2.1134e+07,5.11086e+06,5.45352e+06,1.37568e+06,1.11432e+07,5.22642e+06,5.51673e+06,5.72547e+07,5.28354e+06,1.33250e+06,316575,1.60458e+06,7.42137e+06,171354,...,4.1633e+08,736396,7.84814e+06,6.05377e+07,6996,1.80681e+07,9.548

In [13]:
urban_population.info()
population_growth.info()
population.info()
access_electricity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 65 columns):
Country Name      264 non-null object
Country Code      264 non-null object
Indicator Name    264 non-null object
Indicator Code    264 non-null object
1960              258 non-null float64
1961              258 non-null float64
1962              258 non-null float64
1963              258 non-null float64
1964              258 non-null float64
1965              258 non-null float64
1966              258 non-null float64
1967              258 non-null float64
1968              258 non-null float64
1969              258 non-null float64
1970              258 non-null float64
1971              258 non-null float64
1972              258 non-null float64
1973              258 non-null float64
1974              258 non-null float64
1975              258 non-null float64
1976              258 non-null float64
1977              258 non-null float64
1978              258 non-null float64
19

In [227]:
def clean_all_df(df):
    df_copy = df.copy()
    df_copy = df_copy.drop(df.columns[len(df.columns)-1], axis=1)
    df_copy = df_copy.drop(columns=['Indicator Code','Indicator Name','2019','Country Code'])
    df_copy = df_copy.fillna(method='ffill')
    df_copy = df_copy.T
    df_copy = df_copy.reset_index()
    header = np.array(df_copy.iloc[0])
    header[0] = 'Date'
    df_copy = df_copy[1:]
    df_copy.columns = header
    df_copy = df_copy.melt(id_vars=['Date'], var_name='Country')
    df_copy = df_copy.sort_values('value',ascending=False)
    return df_copy

In [228]:
urban_population_cleaned = clean_all_df(urban_population)
population_growth_cleaned = clean_all_df(population_growth)
population_cleaned = clean_all_df(population)
access_electricity_cleaned = clean_all_df(access_electricity)

In [229]:
urban_population_cleaned

,Date,Country,value
15221,2018,World,4.19636e+09
15220,2017,World,4.11667e+09
15219,2016,World,4.03654e+09
15218,2015,World,3.95655e+09
15217,2014,World,3.87766e+09
...,...,...,...
14341,1964,Tuvalu,974
14340,1963,Tuvalu,940
14339,1962,Tuvalu,908
14338,1961,Tuvalu,877


In [230]:
fig = px.line(population_cleaned, x='Date', y='value',color='Country')
fig.show()

In [231]:
fig = px.line(urban_population_cleaned, x='Date', y='value',color='Country')
fig.show()